# Day1

## 下午-编程基础知识

### **01 python basics**

编译工具 spyder：像matlab pycharm 很常用 VSCode 更通用 后两者支持AI辅助写代码  
Values 布尔值 字符串 整型 浮点型  
Vaiables 等号赋值   
▶️注意不要用keywords来命名变量 不常见的：assert await lembda yield...  
operators 整型 浮点型 +-*/ 布尔值 逻辑判断  
modules 使用库/工具包里的方法 import  
control statements 控制程序的运行  if, for, while  
functions 封装方法 def...return（可以没有）  
**data types**  
list 列表 [] 索引[0] 切割[:] 可修改  
tuples 元组 () 和list类似但不可修改  
dictionaries 字典{}  

**面向对象编程** 万物皆可是对象  
Class类 （eg. 长方形） 方便复用内部方法  
属性  长 宽  
方法  求周长 求面积  
▶️类的继承 （eg.正方形）Class 正方形(长方形)：  
object实例化  

In [2]:
class Linear():  #定义类
    def __init__(self, n_input, n_output):  #属性初始化
        self.n_input = n_input
        self.n_output = n_output
    def compute_n_params(self):  #定义方法
        num_params = self.n_input * self.n_output
        return num_params
layer1 = Linear(10,100)  #对象实例化

print(layer1.compute_n_params())

1000


### **02 Numpy basics**

科学计算包  
数据基本结构：n维数组 N-D array  
array是可修改的，有很多属性，如shape size等  
array broadcasting 广播  
不同size的矩阵的相加 numpy可以自动维度扩增  
vetor operations 点乘  
matrix operations 第一个矩阵的列和第二个矩阵的行是一样长度才能点乘  
operations along axes 规约操作 求和 累加 a.cumsum()  
slicing  
reshape  
matrix operations  
linear alebra  
fourier transform  
random sampling  
distributions in random  
也可以了解一下scipy  

### **03 Brainpy introduction**

**Infrastructures** brainpy有哪些可以调用的方法  
operators 针对array  
>**Main features**  
1)dense operators 针对稠密数组的操作（传统的操作） 
基于jax对numpy进行改造 brainpy.math  
2)dedicated operators ▶️最核心  模拟大脑神经元稀疏连接 提高效率  
sparse connections  
event-driven  
JIT connection  
surrogate gradient DP模型 函数必须是可微的 但是 神经元重制操作是不可微的  
3)numerical integrators  数值积分器   
brainpy.odeint   
brainpy.sdeint 随机积分方程  
brain.fdeint ▶️day2课程细讲  
delayed differential equations  
4)modular and composable  
通过面向对象进行封装 嵌套 体现层级关系 模拟大脑  
iron channel(eg. IK model)--neuron(eg. HH model)--network(eg. E-I balance model)--system   
5)object-oriented JIT compilation 面向对象加速计算  
brain.math.jit/grad/for_loop/ifelse
将jax改编成面向对象  

**function**  
模型建立 仿真 训练（调整参数） 分析（分叉分析...）  
**just-in-time compilation**  jax xla  
**Devices** CPU GPU TPU  


### **04 Brainpy programming basics**

**两种计算机编译方法**  
· static complilation 编译型语言 将code转为机器语言再运行 eg. c c++  
· an interpreter 解释型语言 直接执行source code  eg.python 跑得慢  


#### **just-in-time compilation**  
将程序代码转为二进制机器码  
具体用法： 
1. 对函数进行JIT编译  

In [3]:
import brainpy as bp
import brainpy.math as bm
import numpy as np
import jax.numpy as jnp
bm.set_platform('cpu')

/Users/zhangyimu/anaconda3/envs/bdp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def gelu(x): #举例 高斯误差线性单元函数
    sqrt = bm.sqrt(2/bm.pi)
    cdf = 0.5 * (1.0 + bm.tanh(sqrt * (x + 0.044715 * (x ** 3))))
    y = x * cdf  
    return y
x = bm.random.random(100000)
gelu_jit = bm.jit(gelu)  # 使用JIT
gelu_jit(x)

Array(value=Array([0.5897823 , 0.24811038, 0.02101432, ..., 0.46452346, 0.03164158,
                   0.51636827]),
      dtype=float32)

2. 面向对象进行JIT编译

>**必须要做的事**  
1. 将类对象定义为brainpy.BrainPyObject的子类（继承） 
2. 所有随着时间变化的变量(动态变量) 需要被定义为 brainpy.math.Variable


In [5]:
class LogisticRegression(bp.BrainPyObject): 
    #在逻辑回归分类器中，权重W需要在训练过程中多次修改，故将w设置为动态变量，其他变量则为静态变量
    def __init__(self, dimension):
        super(LogisticRegression, self).__init__  # 继承基类BrainPyObject，可以自动开启jit编译
        #定义参数（属性）
        self.dimension = dimension
        #定义动态变量
        self.w = bm.Variable(2.0 * bm.ones(dimension)-1.3)

    def __call__(self, X, Y):
        u = bm.dot(((1.0/(1.0+bm.exp(-Y*bm.dot(X, self.w)))-1.0)*Y),X)
        self.w.value = self.w - u # in-place update 就地更新动态变量

>**用完jit之后不容易debug，提示很难看懂怎么办？**  
可以关闭JIT compilation (in bp.DSRunner)

#### **data operations** 数据操作
1. arrays（静态数组）

In [6]:
bm_array = bm.array([0,1,2,3,4,5])
bm_array

Array(value=Array([0, 1, 2, 3, 4, 5]), dtype=int32)

>array的属性举例  
.ndim  
.shape  
.size  
.dtype  

In [7]:
# 将BrainPy array转换成Jax array
bm_array.value #查看数组中内部存储的数据类型

Array([0, 1, 2, 3, 4, 5], dtype=int32)

2. Variables (动态变量)  
   动态变量是一个指向数组的指针，指向内存中存储的数组的值（DeviceArray）

In [8]:
t = bm.arange(4)
v = bm.Variable(t) #转换数组为动态变量，可以在JIT编译环境下实现数据的动态修改
v

Variable(value=Array([0, 1, 2, 3]), dtype=int32)

In [9]:
v.value #读取动态变量中存储的值 DeviceArray

Array([0, 1, 2, 3], dtype=int32)

>如何对动态变量进行**就地更新**？（in-place updating）


·切片和索引 indexing and slicing

In [22]:
#索引
v = bm.Variable(bm.arange(4))
v[1] = 10
print(v)
#切片
v[1:3] = 9
print(v)

Variable(value=Array([ 0, 10,  2,  3]), dtype=int32)
Variable(value=Array([0, 9, 9, 3]), dtype=int32)


·增量赋值augmented assignment

In [23]:
v += 1
v

Variable(value=Array([ 1, 10, 10,  4]), dtype=int32)

·**.value赋值** （推荐使用）

In [19]:
v.value = bm.arange(4) 
v

Variable(value=Array([0, 1, 2, 3]), dtype=int32)

> 当新的数组覆盖动态变量的数据时，需要保证数组的形状 元素类型与动态变量完全一致！

·.update赋值（和.value赋值用法一样）

In [24]:
v.update(bm.array([3,4,5,6]))
v

Variable(value=Array([3, 4, 5, 6]), dtype=int32)

#### **control flows** 控制流
1. 条件语句if else

需要注意：当条件语句依赖动态变量时，不可以直接使用python的if else语句  
需要使用以下方法进行替代  

·brainpy.math.where()

In [33]:
class OddEvenWhere(bp.BrainPyObject):
    def __init__(self):
        super(OddEvenWhere, self).__init__()
        self.rand = bm.Variable(bm.random.random(1))
        self.a = bm.Variable(bm.zeros(1))
    def __call__(self):
        #where(condition,x,y) x y为输出参数
        self.a += bm.where(self.rand < 0.5, 1., -1.) #条件判断 若self.rand<0.5,则self.a +1 否则-1
        return self.a
model = OddEvenWhere()
model()

Variable(value=Array([1.]), dtype=float32)

·brainpy.math.ifelse()

In [37]:
# ifelse(conditions, branches, operands) 
# branches 是所有的分支语句，可以是数字也可以是函数
# operands 分支语句是函数时，所需的参数

class OddEvenWhere(bp.BrainPyObject):
    def __init__(self):
        super(OddEvenWhere, self).__init__()
        self.rand = bm.Variable(bm.random.random(1))
        self.a = bm.Variable(bm.zeros(1))
    def __call__(self):
        self.a += bm.ifelse(self.rand[0] < 0.5, [lambda _: 1., lambda _: -1.],_) # 用列表整理多个branch
        #注意：如果没有operands，需要用_占位
        return self.a
model = OddEvenWhere()
model()

Variable(value=Array([1.]), dtype=float32)

2. 循环语句for loop  
python for loop 也能用 但是效率低
可以使用bm.for_loop()函数 提高计算速度


·brainpy.math.for_loop()

In [38]:
class LoopStruct(bp.BrainPyObject):
    def __init__(self):
        super(LoopStruct, self).__init__()
        rng = bm.random.RandomState(123)
        self.seq = rng.random(1000)
        self.res = bm.Variable(bm.zeros(1))
    def __call__(self):
        def add(s):
            self.res += s
            return self.res.value
        return bm.for_loop(body_fun=add, operands=self.seq)
        #for_loop(body_function, xs) 此处xs是可迭代对象，对其中的每一个元素进行function运算
model = LoopStruct()
model()

Array([[3.66994977e-01],
       [1.25954640e+00],
       [2.13131094e+00],
       [2.16719151e+00],
       [3.04240823e+00],
       [3.17834520e+00],
       [3.64254427e+00],
       [4.43797779e+00],
       [4.87824726e+00],
       [5.02111292e+00],
       [5.96435928e+00],
       [6.08199406e+00],
       [6.12994862e+00],
       [6.27706575e+00],
       [6.81935406e+00],
       [6.86844254e+00],
       [7.11615133e+00],
       [7.26757240e+00],
       [8.09881306e+00],
       [8.77181530e+00],
       [9.67675209e+00],
       [1.01394987e+01],
       [1.08685760e+01],
       [1.12544422e+01],
       [1.21589289e+01],
       [1.21974010e+01],
       [1.22036724e+01],
       [1.30537529e+01],
       [1.34715977e+01],
       [1.37603874e+01],
       [1.47282190e+01],
       [1.50743761e+01],
       [1.54980888e+01],
       [1.55543718e+01],
       [1.60150242e+01],
       [1.61848984e+01],
       [1.70158901e+01],
       [1.72855930e+01],
       [1.75178032e+01],
       [1.80801125e+01],


3. while loop

In [60]:
a = bm.Variable(bm.zeros(1)) #在cond_fun 和body_fun中使用的动态变量都需要在dyn_vars中声明
b = bm.Variable(bm.zeros(1))
#条件函数（约束）
def cond_f(x, y): # 在operands里面写入
    return x < 6.
#主体函数
def body_f(x, y):
    a.value += x
    b.value += y
    return x+b[0], y+1

bm.while_loop(body_f, cond_f, dyn_vars=[a, b], operands=(1.,1.))

(Array(11., dtype=float32), Array(4., dtype=float32, weak_type=True))